In [1]:
%matplotlib inline
import seaborn
import quandl
import pandas as pd
import seaborn as sns
import math
from tsmom_model import *
import numpy as np
from datetime import datetime
import statsmodels.formula.api as sm
token="Us3wFmXGgAj_1cUtHAAR"

/Users/mrefermat/anaconda/lib/python2.7/site-packages/IPython/utils/traitlets.py:5: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  warn("IPython.utils.traitlets has moved to a top-level traitlets package.")
/Users/mrefermat/anaconda/lib/python2.7/site-packages/IPython/kernel/__init__.py:13: ShimWarning: The `IPython.kernel` package has been deprecated since IPython 4.0.You should import from ipykernel or jupyter_client instead.
  "You should import from ipykernel or jupyter_client instead.", ShimWarning)


In [2]:
import pylab
sns.set(font_scale=3)
pylab.rcParams['figure.figsize'] = (24, 18)

In [3]:
cleansed=load_amihud_markets_price()

tsmom_model.py:160: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).last()
  data=pd.read_csv('./Price.csv',index_col=0,parse_dates=['Date']).resample(rule='d',how='last')
tsmom_model.py:162: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).last()
  qd=pd.read_csv('./liquid_contracts.csv',index_col=0,parse_dates=['Date']).resample(rule='d',how='last')


In [4]:
amihud=calculate_amihud_liquidity(cleansed)[:'2016'] 

tsmom_model.py:44: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).mean()
  data=pd.read_csv('./Volume.csv',index_col=0,parse_dates=['Date']).resample(rule='d',how='mean')
tsmom_model.py:46: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).sum()
  v=pd.read_csv('./volume_data.csv',index_col=0,parse_dates=['Date']).resample(rule='d',how='sum')
tsmom_model.py:48: FutureWarning: pd.rolling_mean is deprecated for DataFrame and will be removed in a future version, replace with 
	DataFrame.rolling(min_periods=100,window=250,center=False).mean()
  volume=pd.rolling_mean(v,250,min_periods=100).resample(rule='d',how='mean')[:'2016']
tsmom_model.py:48: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).mean()
  volume=pd.rolling_mean(v,250,min_periods=100).resample(rule='d',how='mean')[:'2016']
tsmom_model.py:51: FutureWarning: pd.rolling_mean is deprecated for Series and will be removed in a future ve

In [5]:
sector_rtns={}
d_map=sector_map(amihud)
for sect in d_map.keys():
    sector_rtns[sect]=np.log(cleansed[d_map[sect]].resample(rule='m',how='last')).diff().dropna(how='all')

/Users/mrefermat/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:4: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).last()
  after removing the cwd from sys.path.


# Presentation to be like "Time Varying Liquidity and Momentum Profits (2015) paper

Rank by deciles, HML (8,9,10) - (1,2,3).  Save monthly returns (equal (risk?) weighted) and if you want to annualized then resample.


Then correlation matrix 
* Overall market vol?
* Market illiquidity (median Amihud through time series)


# Amihud (2002) Annual portfolio sorts

In [6]:
sector_zscores=sector_XS_normalized(amihud)

# Annual portfolio sorts

In [7]:
sector_portfolio_sorts={}
for s in sector_zscores.keys():
    sector_portfolio_sorts[s]=quantile_portfolios_annual(sector_zscores[s],cleansed,3)

tsmom_model.py:240: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).median()
  mkts=quantile_columns(rank_data.resample(rule='a',how='median'),year,number_of_buckets,i)
tsmom_model.py:241: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).last()
  rtns = price_data.resample(rule='m',how='last')[mkts].pct_change()[str(y+1)].mean(axis=1)


In [8]:
for sec in sector_portfolio_sorts.keys():
    print(sec)
    print(portfolio_sort_table(sector_portfolio_sorts[sec],np.log(cleansed.resample(rule='m',how='last')).diff().dropna(how='all'),d_map,sec).to_latex())


Agriculturals


/Users/mrefermat/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:3: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).last()
  This is separate from the ipykernel package so we can avoid doing imports until


\begin{tabular}{lrrrr}
\toprule
{} &      0 &      1 &      2 &  Factor \\
\midrule
Monthly Return (in \%)        &   0.77 &   0.64 &   0.96 &    0.20 \\
Standard Deviation           &  18.88 &  13.53 &  16.41 &   16.68 \\
Information Ratio            &   0.49 &   0.57 &   0.70 &    0.14 \\
Skewness                     &   1.10 &  -0.49 &  -0.05 &   -0.43 \\
Excess Kurtosis              &   7.48 &   2.57 &   0.50 &    3.00 \\
CAPM Alpha Annualized (in \%) &   4.85 &   4.65 &   7.99 &    3.14 \\
CAPM Alpha Tstat             &   2.32 &   2.72 &   3.83 &    0.88 \\
CAPM Beta (in \%)             &   1.20 &   0.83 &   0.98 &   -0.22 \\
CAPM Beta Tstat              &  12.17 &  16.74 &  22.59 &   -1.90 \\
\$R\textasciicircum2\$                        &   0.74 &   0.69 &   0.66 &    0.03 \\
\bottomrule
\end{tabular}

All
\begin{tabular}{lrrrr}
\toprule
{} &      0 &      1 &      2 &  Factor \\
\midrule
Monthly Return (in \%)        &   0.39 &   0.55 &   0.65 &    0.26 \\
Standard Deviation   

# FHT Annual portfolio sorts

In [27]:
FHT=calculate_FHT(cleansed)
sector_zscores=sector_XS_normalized(FHT)

In [28]:
sector_portfolio_sorts_FHT={}
for s in sector_zscores.keys():
    sector_portfolio_sorts_FHT[s]=quantile_portfolios_annual(sector_zscores[s].dropna(how='all'),cleansed,3)

In [29]:
for sec in sector_portfolio_sorts_FHT.keys():
    print(sec)
    print(portfolio_sort_table(sector_portfolio_sorts_FHT[sec],np.log(cleansed.resample(rule='m',how='last')).diff().dropna(how='all'),d_map,sec).to_latex())

Agriculturals


/Users/mrefermat/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:3: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).last()
  This is separate from the ipykernel package so we can avoid doing imports until


\begin{tabular}{lrrrr}
\toprule
{} &      0 &      1 &      2 &  Factor \\
\midrule
Monthly Return (in \%)        &   0.34 &   0.62 &   0.64 &    0.30 \\
Standard Deviation           &  13.53 &  15.97 &  17.52 &   17.94 \\
Information Ratio            &   0.30 &   0.47 &   0.44 &    0.20 \\
Skewness                     &  -0.09 &  -0.08 &   0.91 &    0.68 \\
Excess Kurtosis              &   0.94 &   1.23 &   6.34 &    3.26 \\
CAPM Alpha Annualized (in \%) &   4.28 &   6.24 &   5.29 &    1.01 \\
CAPM Alpha Tstat             &   2.18 &   3.36 &   2.06 &    0.26 \\
CAPM Beta (in \%)             &   0.84 &   1.09 &   1.13 &    0.29 \\
CAPM Beta Tstat              &  16.42 &  20.87 &  12.41 &    2.46 \\
\$R\textasciicircum2\$                        &   0.64 &   0.78 &   0.66 &    0.04 \\
\bottomrule
\end{tabular}

All
\begin{tabular}{lrrrr}
\toprule
{} &      0 &      1 &      2 &  Factor \\
\midrule
Monthly Return (in \%)        &   0.26 &   0.55 &   0.61 &    0.35 \\
Standard Deviation   

# Monthly portoflio sorts

In [25]:
amihud_monthly = read_monthly(amihud=True,sorts=3,xs=True)

In [26]:
for sec in amihud_monthly.keys():
    print(sec)
    #print(portfolio_sort_table(amihud_monthly[sec],sector_rtns[sec]).to_latex())
    print(portfolio_sort_table(amihud_monthly[sec],np.log(cleansed.resample(rule='m',how='last')).diff().dropna(how='all'),d_map,sec).to_latex())

Agriculturals


/Users/mrefermat/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:4: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).last()
  after removing the cwd from sys.path.


\begin{tabular}{lrrrr}
\toprule
{} &      0 &      1 &      2 &  Factor \\
\midrule
Monthly Return (in \%)        &   0.49 &   0.75 &   0.98 &    0.49 \\
Standard Deviation           &  16.88 &  14.30 &  19.25 &   17.36 \\
Information Ratio            &   0.35 &   0.63 &   0.61 &    0.34 \\
Skewness                     &   0.09 &  -0.52 &   1.21 &    1.11 \\
Excess Kurtosis              &   0.74 &   2.26 &   8.19 &    6.11 \\
CAPM Alpha Annualized (in \%) &   2.49 &   6.42 &   8.26 &    5.76 \\
CAPM Alpha Tstat             &   1.30 &   2.88 &   3.08 &    1.45 \\
CAPM Beta (in \%)             &   1.07 &   0.82 &   1.11 &    0.04 \\
CAPM Beta Tstat              &  18.59 &  13.58 &  12.85 &    0.36 \\
\$R\textasciicircum2\$                        &   0.77 &   0.63 &   0.64 &    0.00 \\
\bottomrule
\end{tabular}

All
\begin{tabular}{lrrrr}
\toprule
{} &      0 &      1 &      2 &  Factor \\
\midrule
Monthly Return (in \%)        &   0.39 &   0.34 &   0.65 &    0.26 \\
Standard Deviation   

# Monthly FHT

In [22]:
FHT_monthly = read_monthly(amihud=False,sorts=3,xs=True)

In [23]:
for sec in FHT_monthly.keys():
    print(sec)
    #print(portfolio_sort_table(FHT_monthly[sec],sector_rtns[sec]).to_latex())
    print(portfolio_sort_table(FHT_monthly[sec],np.log(cleansed.resample(rule='m',how='last')).diff().dropna(how='all'),d_map,sec).to_latex())



Agriculturals


/Users/mrefermat/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:4: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).last()
  after removing the cwd from sys.path.


\begin{tabular}{lrrrr}
\toprule
{} &      0 &      1 &      2 &  Factor \\
\midrule
Monthly Return (in \%)        &   0.93 &   0.45 &   0.36 &   -0.58 \\
Standard Deviation           &  12.10 &  15.94 &  18.67 &   17.73 \\
Information Ratio            &   0.92 &   0.34 &   0.23 &   -0.39 \\
Skewness                     &   0.16 &  -0.22 &   1.07 &    0.94 \\
Excess Kurtosis              &   1.35 &   0.74 &   7.29 &    5.66 \\
CAPM Alpha Annualized (in \%) &  10.36 &   5.41 &   1.28 &   -9.08 \\
CAPM Alpha Tstat             &   4.78 &   2.74 &   0.54 &   -2.31 \\
CAPM Beta (in \%)             &   0.73 &   1.09 &   1.24 &    0.51 \\
CAPM Beta Tstat              &  14.40 &  22.68 &  13.71 &    4.22 \\
\$R\textasciicircum2\$                        &   0.55 &   0.75 &   0.70 &    0.14 \\
\bottomrule
\end{tabular}

All
\begin{tabular}{lrrrr}
\toprule
{} &      0 &      1 &      2 &  Factor \\
\midrule
Monthly Return (in \%)        &   0.41 &   0.44 &   0.59 &    0.18 \\
Standard Deviation   

In [30]:
# Spanning Test
